In [1]:
# make external scripts auto reload
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
from lightgbm.sklearn import LGBMRanker

from experiment_template import *
from candidate_generation import *

In [16]:
BASE_PATH = '../../../data/'
DATA_PATH = BASE_PATH + 'sample_0.05/'
# DATA_PATH = BASE_PATH + 'parquet/'

# make sure the same data preprocessing as in the radek notebook have been performed
# (see 02 FE/DataProcessingRadek.ipynb)
transactions = pd.read_parquet(DATA_PATH + 'transactions_train.parquet')
customers = pd.read_parquet(DATA_PATH + 'customers.parquet')
articles = pd.read_parquet(DATA_PATH + 'articles.parquet')
sample_submission = pd.read_csv(BASE_PATH + 'original/sample_submission.csv')

In [ ]:
# def get_data(data, test_week):
#     # candidates = []

#     # ### combine
#     # result = data.copy()[['week', 'customer_id', 'article_id']]
#     # result['purchased'] = True
#     # result = pd.concat([
#     #     result, candidates
#     # ])
#     # result.purchased.fillna(False, inplace=True)
#     # result.drop_duplicates(['customer_id', 'article_id', 'week'], inplace=True)
#     # result.sort_values(['week', 'customer_id'], inplace=True)
#     # result.reset_index(drop=True, inplace=True)
#     # return result

#     ### repurchase
#     # each week is seen as a basket
#     # the items bought in one basket, will be example for the next basket
#     # the items bought in the last basket, will be candidates for the test basket
#     c2weeks = data.groupby('customer_id')['week'].unique()
#     c2weeks2shifted_weeks = {}
#     for c_id, weeks in c2weeks.items():
#         c2weeks2shifted_weeks[c_id] = {}
#         for i in range(weeks.shape[0]-1):
#             c2weeks2shifted_weeks[c_id][weeks[i]] = weeks[i+1]
#         c2weeks2shifted_weeks[c_id][weeks[-1]] = test_week
#     candidates_last_purchase = data.copy()
#     weeks = []
#     for i, (c_id, week) in enumerate(zip(data['customer_id'], data['week'])):
#         weeks.append(c2weeks2shifted_weeks[c_id][week])
#     candidates_last_purchase.week=weeks

#     ### bestseller
#     # if a user bought an item in a given week, the 12 most popular items in the previous week are example for that week
#     # the best selling items in the last week are candidates for all users
#     mean_price = data \
#         .groupby(['week', 'article_id'])['price'].mean()
#     sales = data \
#         .groupby('week')['article_id'].value_counts() \
#         .groupby('week').rank(method='dense', ascending=False) \
#         .groupby('week').head(12).rename('bestseller_rank').astype('int8')
#     bestsellers_previous_week = pd.merge(sales, mean_price, on=['week', 'article_id']).reset_index()
#     bestsellers_previous_week.week += 1
#     unique_transactions = data \
#         .groupby(['week', 'customer_id']) \
#         .head(1) \
#         .drop(columns=['article_id', 'price']) \
#         .copy()
#     candidates_bestsellers = pd.merge(
#         unique_transactions,
#         bestsellers_previous_week,
#         on='week',
#     )
#     test_set_transactions = unique_transactions.drop_duplicates('customer_id').reset_index(drop=True)
#     test_set_transactions.week = test_week
#     candidates_bestsellers_test_week = pd.merge(
#         test_set_transactions,
#         bestsellers_previous_week,
#         on='week'
#     )
#     candidates_bestsellers = pd.concat([candidates_bestsellers, candidates_bestsellers_test_week])
#     candidates_bestsellers.drop(columns='bestseller_rank', inplace=True)

#     ### combine
#     d = data.copy()
#     d['purchased'] = True
    
#     result = pd.concat([
#         d, candidates_last_purchase, candidates_bestsellers
#     ])
#     result.purchased.fillna(False, inplace=True)
#     result.drop_duplicates(['customer_id', 'article_id', 'week'], inplace=True)
#     result.sort_values(['week', 'customer_id'], inplace=True)
#     result.reset_index(drop=True, inplace=True)
#     return result
    


# def get_examples(data, test_week):
#     data = get_data(data, test_week)
#     return data[data.week != test_week]

# def get_candidates(data, test_week, c):
#     data = get_data(data, test_week)
#     return data[data.week == test_week]

# def get_examples(data, test_week):
#     c =  pd.Series(data.customer_id.unique(), name='customer_id')
#     examples_positive = repurchase(data, c)
#     examples_positive['purchased'] = True
    
#     examples_negative = merge_candidates([
#         popular_similar_items(data, c, articles, 'prod_name', 20, 6, 2),
#         popular_by_feature(data, c, customers, 'postal_code', 15),
#         popular_by_feature(data, c, customers, 'age', 15),
#     ])
#     examples_negative['purchased'] = False

#     examples = pd.concat([examples_positive, examples_negative])
#     examples.drop_duplicates(["customer_id", "article_id"], inplace=True)
#     examples.sort_values(['customer_id'], inplace=True)
#     examples.reset_index(drop=True, inplace=True)
#     return examples


# def get_examples(data, test_week):
#     c = pd.Series(data.customer_id.unique(), name='customer_id')
#     shifted_data = get_shifted_weeks(data)
    
#     candidates1 = popular_similar_items_by_week(shifted_data, articles, 'prod_name', k1=20, k2=6, t=1)
#     # candidates2 = merge_candidates([
#     #     popular_by_feature(data, c, customers, 'postal_code', 5),
#     #     popular_by_feature(data, c, customers, 'age', 5)
#     # ])
#     # candidates2 = pd.merge(
#     #     train_data.drop_duplicates(['customer_id', 'week'])[['customer_id', 'week']],
#     #     candidates2,
#     #     on='customer_id'
#     # )

#     ### combine
#     result = data.copy()[['week', 'customer_id', 'article_id']]
#     result['purchased'] = True
#     result = pd.concat([
#         result, shifted_data, candidates1
#     ])
#     result.purchased.fillna(False, inplace=True)

#     result.pop_rank.fillna(999, inplace=True)
#     ranks = result.groupby(['customer_id', 'article_id', 'week']).pop_rank.min()
#     result = pd.merge(result.drop(columns='pop_rank'), ranks, how='left', on=['customer_id', 'article_id', 'week'])
    
#     result.drop_duplicates(['customer_id', 'article_id', 'week'], inplace=True)
#     result.sort_values(['week', 'customer_id'], inplace=True)
#     result.reset_index(drop=True, inplace=True)
#     return result

# def get_candidates(data, test_week, c):
#     customer_last_week = (
#         data[['customer_id', 'week']]
#         .sort_values(['week'], ascending=False)
#         .drop_duplicates(['customer_id'])
#     )

#     d = pd.merge(customer_last_week, data, how='left', on=['customer_id', 'week'])
    
#     candidates = pd.concat([
#         repurchase(d, c),
#         popular_similar_items(d, c, articles, 'prod_name', 20, 6, 1),
#         # popular_by_feature(d, c, customers, 'postal_code', 15),
#         # popular_by_feature(d, c, customers, 'age', 15),
#     ])
    
#     # handle missing pop rank
#     candidates.pop_rank.fillna(999, inplace=True)
#     ranks = candidates.groupby(['customer_id', 'article_id']).pop_rank.min()
#     candidates = pd.merge(candidates.drop(columns='pop_rank'), ranks, how='left', on=['customer_id', 'article_id'])
    
#     candidates.drop_duplicates(["customer_id", "article_id"], inplace=True)
#     return candidates

In [62]:
### split into training and testing
# one week is used for testing
# a number of weeks leading up to the test week are used to train the ranker
test_week = 104
num_training_weeks = 10
testing_weeks = np.arange(test_week-num_training_weeks, test_week)
train_data = transactions[transactions.week.isin(testing_weeks)].reset_index(drop=True)


### generate training examples and testing candidates
# optimisation: only generate testing candidates for customers with ground truth data
test_customers = None
if test_week < transactions.week.max() + 1:
    p = get_purchases(transactions[transactions.week == test_week])
    test_customers = p.customer_id.values

# get the examples and candidates
# examples are (customer, week, article, purchased)
# candidates are (customer, article)
train_examples, test_candidates = get_examples_candidates(train_data, test_week, test_customers, customers, articles)

# add features and prepare data for ranker
X_train = add_features(train_examples, transactions, customers, articles)
X_test = add_features(test_candidates, transactions, customers, articles)
Y_train = train_examples['purchased']


### fit ranker
# training_groups tells LGBM that each (week, customer_id) combination is a seperate basket
# !!! it is important that the training_examples are sorted according to week, customer_id for this to work
ranker = LGBMRanker(
    force_row_wise=True,
    objective="lambdarank",
    metric="ndcg",
    boosting_type="dart",
    n_estimators=1,
    importance_type='gain',
    verbose=10
)
# train_groups = train_examples.groupby(['customer_id'])['article_id'].count().values
train_groups = train_examples.groupby(['week', 'customer_id'])['article_id'].count().values
ranker.fit(X_train, Y_train, group=train_groups)
print_importance(ranker, X_train.columns)


### evaluate / submit
# generate recommendations
predictions = get_predictions(test_candidates, X_test, ranker, 12)

# fill missing predictions for all customers with popular items in last week
popular = transactions[transactions.week == test_week-1].article_id.value_counts().head(12).index.values
predictions = fill_missing_predictions(predictions, customers.customer_id, popular)

if test_week < transactions.week.max() + 1:
    # calculate score
    purchases = get_purchases(transactions[transactions.week == test_week])
    score = mean_average_precision(predictions, purchases, 12)
    print(score)
else:
    # write submission
    sub = create_submission(predictions, sample_submission)
    sub.to_csv(BASE_PATH + 'sub1.csv.gz', index=False)

[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.201209
[LightGBM] [Info] Total Bins 1233
[LightGBM] [Info] Number of data points in the train set: 1170880, number of used features: 19
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
                 article_price 0.2904586982
              garment_group_no 0.2375597886
               product_type_no 0.2357712120
                    section_no 0.0941273233
             colour_group_code 0.0579860491
                 department_no 0.0372726428
               preferred_price 0.0200650552
       graphical_appearance_no 0.0187174667
                    index_code 0.0080417639
                   postal_code 0.0000000000
                           age 0.0000000000
        fashion_news_frequency 0.0000000000
            club_member_status 0.0000000000
                        Active 0.0000000000
                            FN 0.0000000000
                index_group_no 0.0000000000
    perceived_colour_mas

In [58]:
print(len(train_examples) / len(train_examples.customer_id.unique()))
print(len(test_candidates) / len(test_candidates.customer_id.unique()))
# Radek: 27/user, 15/user

45.35801789401239
9.090310485161645


+ no besteseller: 0.011969058236222092
+ no last week removal: 0.012013546937047771
+ new candidates: 0.008312653896431705
    + candidate generation should match example generation or ranker gets confused
+ no article id: 0.012958022020249708
+ new candidates: 0.009308090077886499
    + slightly better if you don't include article_id as feature
+ new examples, not by week: 0.010802228837859033
+ with article id: 0.01146378936774091
+ repurchase + popular per index_name (per week): 0.012809616546360366
+ with user related candidates: 0.00950611747619205
+ 10 estimators, without user related candidates again: 0.01281175309739873
+ no article id: 0.011082918911172218
+ example generation prod_code instead of index: 0.012408471903703802
+ more restricted candidate generation: 0.014533045459703063
    +  candidate generation should match example generation as much as possible
    +  more candidates than are used as example does not work
+ exactly same candidate as example generation: 0.023595130728855555
+ added pop_rank: 0.02427475386559533
+ 10 estimators: 0.024288773237094562